# Fibre packer demo - how to control misalignment?

*Author: Vedrana Andersen Dahl (vand@dtu.dk)*

THIS IS NOT FINISHED!!!

In [ ]:
import os
import fibre_packer as fp

## Setting up - the same as before

In [ ]:
domain_radius = 70  # Domain radius
fibre_radius_mean = 2  # Mean fibre radius
fibre_radius_sigma = 0.1 * fibre_radius_mean  # Standard deviation of fibre radius
fibre_volume_fraction = 65  # Desired fibre volume fraction
number_slices = 20 # Number of slices to generate


In [ ]:
fib = fp.from_fvf(domain_radius, fibre_volume_fraction, fibre_radius_mean, fibre_radius_sigma)
fib.initialize_start_slice()
losses = fib.optimize_slice_heuristic('start', iters=100, repetitions=3)
fib.show_slice('start','Optimized start slice')

## Misalignment control

Misalignment is primarily controlled by the method `initialize_end_slice(misalignment, k=3)` which initializes the fibre positions in the last slice of the volume. To model misalignment, the fibre positions in the last slice are different than the positions in the first slice. This is done by first copying positions from the first to the last slice, and then performing a series of transformations affecting the positions in the last slice. The method preforms following operations:
- identifies k fibre clusters based on their positions (not all fibres are clustered), 
- independently rotates each of the fibre clusters around the cluster center
- rotates all fibre positions around the centre of the region for a certain angle
- for a subset of fibres swaps their position with the position of the nearby fibres
- adds noise to all fibre positions


Parameters for misalignment
- `angle_range_clusters` a tuple containing three elements (angle_min, angle_max, randomize_sign), and angles given in degrees. For each cluster, a random angle value is be drawn with magnitude between angle_min and angle_max, while rotation direction may or may not be randomized. 
- `angle_range_bundle` a tuple containing three elements (angle_min, angle_max, randomize_sign). Similar as for clusters, a random angle value is drawn for the whole bundle.  
- `swap` a tuple containing three elements (fraction, neighborhood_size). Fraction is fraction of fibres that will be swapped, while neighborhood size is the number of nearest neighbors that are swap candidates. For example (0.01, 1) will randomly select 1% of fibres and swap each with its closest neighbor.
- `noise` a float sigma given in mean radius. Sigma of 1 means that fibre positions are perturbed with values drawn from normal distribution with standard deviation equal to mean fibre radius. 



In [ ]:

misalignment = {
    'angle_range_clusters': (90, 135, True),
    'angle_range_bundle': (135, 180, True), 
    'swap': (0.5, 24),
    'noise': 5
}

fib.initialize_end_slice(misalignment)
fib.show_slice('end','Initial end slice')

In [ ]:
# losses = fib.optimize_slice_heuristic('end', iters=100, repetitions=10)
# fib.show_slice('end','Optimized end slice', show_issues=True)

## Interpolating and optimizing trajectories


Misalignment is also affected by `interpolate_configuration(number_slices)` method, which given the first and the last slice interpolates fibre positions for the slices inbetween, and can do it in several ways: linear, logistic, mixed or staggered.  


In [ ]:
fib.interpolate_configuration(number_slices)
fib.show_3D_configuration('Interpolated configuration')

In [ ]:
fib.animate_slices('Interpolated configuration')